In [4]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from pathlib import Path 

In [5]:
PROCESSED_DATA=Path("../data/processed/cleaned_train.csv").resolve()

In [6]:
df=pd.read_csv(PROCESSED_DATA)

In [7]:
df.head()

road_type  num_lanes  curvature  speed_limit  lighting weather  \
0     urban          2       0.06           35  daylight   rainy   
1     urban          4       0.99           35  daylight   clear   
2     rural          4       0.63           70       dim   clear   
3   highway          4       0.07           35       dim   rainy   
4     rural          1       0.58           60  daylight   foggy   

   road_signs_present  public_road time_of_day  holiday  school_season  \
0               False         True   afternoon    False           True   
1                True        False     evening     True           True   
2               False         True     morning     True          False   
3                True         True     morning    False          False   
4               False        False     evening     True          False   

   num_reported_accidents  accident_risk  
0                       1           0.13  
1                       0           0.35  
2                       2           0.30  
3                       1           0.21  
4                       1           0.56

In [8]:
num_cols = df.select_dtypes(include=np.number).columns.to_list()
cat_cols = df.select_dtypes(exclude=np.number).columns.to_list()

In [27]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import cross_val_score 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [17]:
num_cols.remove("accident_risk")
num_cols

['num_lanes', 'curvature', 'speed_limit', 'num_reported_accidents']

In [28]:
processor = ColumnTransformer([
    ("categorical",OneHotEncoder(handle_unknown="ignore",sparse_output=False),cat_cols),
    ("numerical",StandardScaler(with_mean=False),num_cols)
    
])

In [11]:
X,y= df[df.columns[:-1]],df[df.columns[-1]]

In [35]:
from sklearn.svm import SVR 
from sklearn.metrics import root_mean_squared_error

In [36]:
pipeline=Pipeline([
    ('processor',processor),
    ('predictor',SVR(kernel='rbf'))
])

In [47]:
X_train=X.iloc[:50000]
X_test=X.iloc[50000:52000]
y_train=y.iloc[:50000]
y_test=y.iloc[50000:52000]


In [48]:
scores =cross_val_score(pipeline,X_train,y_train,cv=4,scoring='neg_root_mean_squared_error',n_jobs=3)
scores

array([-0.06212603, -0.06313165, -0.06311737, -0.06355305])

In [49]:
np.mean(scores)

np.float64(-0.06298202463568744)

In [51]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('processor',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['road_type', 'lighting',
                                                   'weather',
                                                   'road_signs_present',
                                                   'public_road', 'time_of_day',
                                                   'holiday',
                                                   'school_season']),
                                                 ('numerical',
                                                  StandardScaler(with_mean=False),
                                                  ['num_lanes', 'curvature',
                                                   'speed_limit',
                                                   'num_reported_accidents'])])),
                ('predictor', SVR())])

In [52]:
root_mean_squared_error(y_test,pipeline.predict(X_test))

0.06283174670338906